In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv', index_col=0)
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv', index_col=0)

In [ ]:
train.head()

# check data

In [ ]:
cols_with_missing_train = [col for col in train.columns if train[col].isnull().any()]
cols_with_missing_test = [col for col in test.columns if test[col].isnull().any()]
print('train data missing vlue is : ',cols_with_missing_train)
print('test data missing vlue is : ',cols_with_missing_test)
print('train data shape is :',train.shape)
print('test data shape is :',test.shape)

there is no missing value

In [ ]:
s = (train.dtypes == 'object')
f = (train.dtypes == 'float')
object_cols = list(s[s].index)
integer_cols = list(f[f].index)
print('object colum is :',object_cols)
print('number of float colum is :',len(integer_cols))

training data has only float data except target colum

train data and test data has almost same

In [ ]:
label = train.groupby('target').sum()
label = label.index

print('kind of target is:',len(label))

plt.figure(figsize=(8,8))
plt.pie(train['target'].value_counts(),counterclock=False, startangle=90,labels=label,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

the number of each target is smae ratio

In [ ]:
# Features with cardinality < 30
df_c=pd.concat([train,test])
c_list = train.nunique()[train.nunique() < 30][:-1]
print('categorical column is')
print(c_list)

10 columns is Categorical

this way to check categorical column is from https://www.kaggle.com/maxencefzr/tps-feb22-eda-extratrees

thanks https://www.kaggle.com/maxencefzr

# add some columns

In [ ]:
continus_train=train.drop(c_list.index,axis=1)
continus_train=continus_train.drop('target',axis=1)
train['min'] = continus_train.min(axis=1)
train['max'] = continus_train.max(axis=1)
train['std'] = continus_train.std(axis=1)

continus_test=test.drop(c_list.index,axis=1)
test['min'] = continus_test.min(axis=1)
test['max'] = continus_test.max(axis=1)
test['std'] = continus_test.std(axis=1)

In [ ]:
train.describe().T

In [ ]:
test.describe().T

# check duplicates

this way to dumplicates and handling it is from https://www.kaggle.com/adamml/classify-bacteria-species-with-acc-98-tps-feb22/notebook

thanks https://www.kaggle.com/adamml

In [ ]:
print('duplicates count is :',train.duplicated().sum())

vc = train.value_counts()
print('unique count is :',len(vc))

In [ ]:
train = pd.DataFrame([list(tup) for tup in vc.index.values], columns=train.columns)
train['sample_weight'] = vc.values
weight = train['sample_weight']
train=train.drop('sample_weight',axis=1)
train.head()

In [ ]:
weight.head()

# Create model

In [ ]:
clf = ExtraTreesClassifier()
X=train.drop('target',axis=1)
y = train.target

clf.fit(X, y,weight)
importances = pd.Series(clf.feature_importances_, index = X.columns)
importances_list = importances.sort_values().index.tolist()

In [ ]:
X.shape

In [ ]:
drop_list = []
score_list=[]
old_sum=0
X_=pd.DataFrame()

for index in range(289):
    X_[importances_list[index]]=X[importances_list[index]]
    sum_ = 0
    kf = StratifiedKFold(n_splits=5)
    for train_index, test_index in kf.split(X_, y):
        train_x = X_.iloc[train_index]
        train_y = y.iloc[train_index]
        train_weight = weight.iloc[train_index]
        valid_x = X_.iloc[test_index]
        valid_y = y.iloc[test_index]
        valid_weight = weight.iloc[test_index]
        clf.fit(train_x, train_y,train_weight)
        prediction=clf.predict(valid_x)
        sum_ = sum_ + accuracy_score(prediction,valid_y)
    score_list.append(sum_*100/5)
    print('score is',sum_*100/5,'%')
    if old_sum>sum_:
        X_=X_.drop(importances_list[index],axis=1)
    else:
        old_sum=sum_

print('list shape is ',X_.shape)
print('list columns is' ,X_.columns)

In [ ]:
df_score_list=pd.DataFrame(score_list)

sns.relplot( data=df_score_list, kind='line',height=6.5, aspect=2)

plt.xlabel('count')
plt.ylabel('score')
plt.show()

In [ ]:
clf.fit(X_, y,weight)
plt.figure(figsize=(15,5))

importances = pd.Series(clf.feature_importances_, index = X_.columns)

importances = importances.sort_values()
importances = importances.tail(30)
importances.plot(kind = "barh")
plt.title("importance in the ExtraTreesClassifier Model")

plt.show()

In [ ]:
prediction = clf.predict(X_)
print('score is',accuracy_score(prediction,y)*100,'%')

# create prediction for submit

In [ ]:
train_list=X_.columns
submission['row_id']=test.index
submission['target']=clf.predict(test[train_list])
submission.head()

In [ ]:
label = submission.groupby('target').sum()
label = label.index

print('kind of submission target is:',len(label))

plt.figure(figsize=(8,8))
plt.pie(submission['target'].value_counts(),counterclock=False, startangle=90,labels=label,
       autopct='%1.1f%%', pctdistance=0.7)
 
plt.show()

In [ ]:
submission.to_csv('submission.csv', index=False)